Memory is all about remembering about the context that what we have been talking about in previous prompts

In [ ]:
!pip install langchain
!pip install langchain-core
!pip install langchain_openai
!pip install langchain_community

In [3]:
import os
os.environ["OPENAI_API_KEY"] = ""

#### Conversation Buffer Memory

In [ ]:
from langchain import PromptTemplate
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI, ChatOpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

prompt_template = PromptTemplate(
    input_variables =["history", "new_input"],
    template = """You are having a chat with a human.
    previous chat: {history}
    Human: {new_input}
    Assistant: """
)

memory = ConversationBufferMemory()

#llm = ChatOpenAI()
llm = OpenAI()

chain = LLMChain(llm=llm, prompt=prompt_template, memory = memory, verbose=True)

chain.invoke({"new_input": "Which country is the biggest exporter of cotton?"})



> Entering new LLMChain chain...
Prompt after formatting:
You are having a chat with a human.
    previous chat: 
    Human: Which country is the biggest exporter of cotton?
    Assistant: 

> Finished chain.


{'new_input': 'Which country is the biggest exporter of cotton?',
 'history': '',
 'text': 'The biggest exporter of cotton is currently India.'}

In [ ]:
chain.invoke({"new_input": "What is the total value of cotton exports of this country"})



> Entering new LLMChain chain...
Prompt after formatting:
You are having a chat with a human.
    previous chat: Human: Which country is the biggest exporter of cotton?
AI: The biggest exporter of cotton is currently India.
    Human: What is the total value of cotton exports of this country
    Assistant: 

> Finished chain.


{'new_input': 'What is the total value of cotton exports of this country',
 'history': 'Human: Which country is the biggest exporter of cotton?\nAI: The biggest exporter of cotton is currently India.',
 'text': 'In 2019, India exported cotton worth $7.5 billion. '}

In [ ]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_openai import ChatOpenAI

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are having a chat with a human."
        ),
        MessagesPlaceholder(
            variable_name="history"
        ),
        HumanMessagePromptTemplate.from_template(
            "{new_input}"
        ),
    ]
)

chat_memory = ConversationBufferMemory(return_messages=True)

chat_llm = ChatOpenAI()

chat_chain = LLMChain(llm=chat_llm, prompt=chat_prompt, memory = chat_memory, verbose=True)

chat_chain.invoke({"new_input": "Which country is the biggest exporter of cotton?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are having a chat with a human.
Human: Which country is the biggest exporter of cotton?

> Finished chain.


{'new_input': 'Which country is the biggest exporter of cotton?',
 'history': [HumanMessage(content='Which country is the biggest exporter of cotton?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The largest exporter of cotton in the world is India. India exports a significant amount of cotton to various countries around the world.', additional_kwargs={}, response_metadata={})],
 'text': 'The largest exporter of cotton in the world is India. India exports a significant amount of cotton to various countries around the world.'}

#### Changing the memory variable name

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are having a chat with a human."
        ),
        MessagesPlaceholder(
            variable_name="prev_conv"
        ),
        HumanMessagePromptTemplate.from_template(
            "{new_input}"
        ),
    ]
)

chat_memory = ConversationBufferMemory(memory_key="prev_conv", return_messages=True)

chat_llm = ChatOpenAI()

chat_chain = LLMChain(llm=chat_llm, prompt=chat_prompt, memory = chat_memory, verbose=True)

chat_chain.invoke({"new_input": "Which country is the biggest exporter of cotton?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are having a chat with a human.
Human: Which country is the biggest exporter of cotton?

> Finished chain.


{'new_input': 'Which country is the biggest exporter of cotton?',
 'prev_conv': [HumanMessage(content='Which country is the biggest exporter of cotton?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The biggest exporter of cotton is India. India is a major producer and exporter of cotton, with a significant share of the global cotton market. Other major cotton exporting countries include the United States and Brazil.', additional_kwargs={}, response_metadata={})],
 'text': 'The biggest exporter of cotton is India. India is a major producer and exporter of cotton, with a significant share of the global cotton market. Other major cotton exporting countries include the United States and Brazil.'}

#### Adding messages to memory

In [ ]:
sample_memory = ConversationBufferMemory(memory_key="history")

In [ ]:
sample_memory.load_memory_variables({})

{'history': ''}

In [ ]:
sample_memory.chat_memory.add_user_message("Which country is the biggest exporter of cotton?")
sample_memory.chat_memory.add_ai_message("China")

In [ ]:
sample_memory.load_memory_variables({})

{'history': 'Human: Which country is the biggest exporter of cotton?\nAI: China'}

In [ ]:
from langchain import PromptTemplate
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

prompt_template = PromptTemplate(
    input_variables =["history", "new_input"],
    template = """You are having a chat with a human.
    previous chat: {history}
    Human: {new_input}
    Assistant: """
)


llm = ChatOpenAI()

chain = LLMChain(llm=llm, prompt=prompt_template, memory = sample_memory, verbose=True)


In [ ]:
chain.invoke({"new_input": "What is the total value of cotton exports of this country"})



> Entering new LLMChain chain...
Prompt after formatting:
You are having a chat with a human.
    previous chat: Human: Which country is the biggest exporter of cotton?
AI: China
    Human: What is the total value of cotton exports of this country
    Assistant: 

> Finished chain.


{'new_input': 'What is the total value of cotton exports of this country',
 'history': 'Human: Which country is the biggest exporter of cotton?\nAI: China',
 'text': 'I am not sure about the exact value, would you like me to look it up for you?'}

#### Conversation Chain instead of LLMChain

In [ ]:
from langchain import PromptTemplate
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

from langchain.chains import ConversationChain

memory = ConversationBufferMemory(memory_key="history")

llm = ChatOpenAI()

chain = ConversationChain(llm=llm, memory = memory, verbose=True)

chain.invoke({"input": "Which country is the biggest exporter of cotton?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Which country is the biggest exporter of cotton?
AI:

> Finished chain.


{'input': 'Which country is the biggest exporter of cotton?',
 'history': '',
 'response': "The biggest exporter of cotton is currently India. India is one of the world's largest producers and exporters of cotton, with around 26% of the global cotton exports coming from India. The country's climate and agricultural practices make it ideal for cotton cultivation, leading to significant export volumes each year."}

In [ ]:
print(chain.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [ ]:
chain.invoke({"input": "What is the total value of cotton exports of this country"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Which country is the biggest exporter of cotton?
AI: The biggest exporter of cotton is currently India. India is one of the world's largest producers and exporters of cotton, with around 26% of the global cotton exports coming from India. The country's climate and agricultural practices make it ideal for cotton cultivation, leading to significant export volumes each year.
Human: What is the total value of cotton exports of this country
AI:

> Finished chain.


{'input': 'What is the total value of cotton exports of this country',
 'history': "Human: Which country is the biggest exporter of cotton?\nAI: The biggest exporter of cotton is currently India. India is one of the world's largest producers and exporters of cotton, with around 26% of the global cotton exports coming from India. The country's climate and agricultural practices make it ideal for cotton cultivation, leading to significant export volumes each year.",
 'response': "In the fiscal year ending in March 2021, India's total value of cotton exports was approximately $7.66 billion USD. This accounted for a significant portion of India's overall textile exports and contributed significantly to the country's economy."}

In [ ]:
print(chain.memory.buffer)

Human: Which country is the biggest exporter of cotton?
AI: The biggest exporter of cotton is currently India. India is one of the world's largest producers and exporters of cotton, with around 26% of the global cotton exports coming from India. The country's climate and agricultural practices make it ideal for cotton cultivation, leading to significant export volumes each year.
Human: What is the total value of cotton exports of this country
AI: In the fiscal year ending in March 2021, India's total value of cotton exports was approximately $7.66 billion USD. This accounted for a significant portion of India's overall textile exports and contributed significantly to the country's economy.


#### ConversationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

memory = ConversationBufferWindowMemory(k=1) # remember only last chat

llm = ChatOpenAI()

convo_chain = ConversationChain(llm=llm, memory=memory)

convo_chain.invoke({"input": "Which country is the biggest exporter of cotton?"})

{'input': 'Which country is the biggest exporter of cotton?',
 'history': '',
 'response': 'The largest exporter of cotton in the world is India. They produce about 26% of the total cotton exported globally. China and the United States are also major exporters, with China being the largest consumer of cotton. Would you like to know more about cotton production?'}

In [ ]:
convo_chain.invoke({"input": "What is the total value of cotton exports of this country"})

{'input': 'What is the total value of cotton exports of this country',
 'history': 'Human: Which country is the biggest exporter of cotton?\nAI: The largest exporter of cotton in the world is India. They produce about 26% of the total cotton exported globally. China and the United States are also major exporters, with China being the largest consumer of cotton. Would you like to know more about cotton production?',
 'response': "India's total value of cotton exports is approximately $8 billion USD. This makes cotton one of India's most valuable export commodities. Would you like to know more about the cotton industry in India?"}

In [ ]:
print(convo_chain.memory.buffer)

Human: What is the total value of cotton exports of this country
AI: India's total value of cotton exports is approximately $8 billion USD. This makes cotton one of India's most valuable export commodities. Would you like to know more about the cotton industry in India?


#### ConversationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

memory = ConversationBufferWindowMemory(k=1)

llm = ChatOpenAI()

convo_chain = ConversationChain(llm=llm, memory=memory)

convo_chain.invoke({"input": "Which country is the biggest exporter of cotton?"})

{'input': 'Which country is the biggest exporter of cotton?',
 'history': '',
 'response': "The biggest exporter of cotton is India, followed by the United States and China. India is known for its high-quality cotton production, particularly in states like Gujarat and Maharashtra. The country's cotton industry plays a significant role in its economy, with exports to various countries around the world."}

In [ ]:
convo_chain.invoke({"input": "What is the total value of cotton exports of this country"})

{'input': 'What is the total value of cotton exports of this country',
 'history': "Human: Which country is the biggest exporter of cotton?\nAI: The biggest exporter of cotton is India, followed by the United States and China. India is known for its high-quality cotton production, particularly in states like Gujarat and Maharashtra. The country's cotton industry plays a significant role in its economy, with exports to various countries around the world.",
 'response': "According to data from the International Trade Centre, India's total value of cotton exports in 2020 was approximately $5.9 billion. This makes India one of the largest exporters of cotton globally."}

In [ ]:
print(convo_chain.memory.buffer)

Human: What is the total value of cotton exports of this country
AI: According to data from the International Trade Centre, India's total value of cotton exports in 2020 was approximately $5.9 billion. This makes India one of the largest exporters of cotton globally.


#### ConversationSummaryMemory

In [ ]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import OpenAI

memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "Which country is the biggest exporter of cotton?"}, {"output": "China"})

/tmp/ipython-input-44-2845879327.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))


In [ ]:
memory.load_memory_variables({})

{'history': '\nThe human asks which country is the biggest exporter of cotton. The AI responds with China.'}

In [ ]:
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
llm = OpenAI(temperature=0)
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=OpenAI()),
    verbose=True
)
conversation_with_summary.invoke(input="Which country is the biggest exporter of cotton?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Which country is the biggest exporter of cotton?
AI:

> Finished chain.


{'input': 'Which country is the biggest exporter of cotton?',
 'history': '',
 'response': " According to the latest data from the World Trade Organization, China is currently the biggest exporter of cotton, accounting for approximately 26% of the global cotton exports. Other major exporters include India, the United States, and Pakistan. However, it's worth noting that the top exporters can vary from year to year depending on factors such as weather conditions and market demand. Is there anything else you would like to know about cotton exports?"}

In [ ]:
conversation_with_summary.invoke(input="What is the total value of cotton exports of this country")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
 The human asks about the biggest exporter of cotton and the AI provides information about current data from the World Trade Organization. China is currently the biggest exporter, followed by India, the United States, and Pakistan. However, the top exporters can vary based on various factors. The AI also asks if there is any additional information the human would like to know about cotton exports.
Human: What is the total value of cotton exports of this country
AI:

> Finished chain.


{'input': 'What is the total value of cotton exports of this country',
 'history': ' The human asks about the biggest exporter of cotton and the AI provides information about current data from the World Trade Organization. China is currently the biggest exporter, followed by India, the United States, and Pakistan. However, the top exporters can vary based on various factors. The AI also asks if there is any additional information the human would like to know about cotton exports.',
 'response': " According to the latest data from the World Trade Organization, the total value of cotton exports for China in 2020 was approximately $15.8 billion. This accounted for 26.7% of the global cotton export market. India followed closely behind with a total value of $10.8 billion, accounting for 18.2% of the market. The United States had a total value of $5.9 billion, making up 9.9% of the market, and Pakistan had a total value of $3.9 billion, accounting for 6.6% of the market. However, it's impor

#### Runnable with Message History with session ID

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are having a chat with a human.",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
chain = prompt | model | StrOutputParser()

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


runnable = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [ ]:
runnable.invoke(
    {"input": "Which country is the biggest exporter of cotton?"},
    config={"configurable": {"session_id": "sess1"}},
)

'The biggest exporter of cotton in the world is the United States, followed by India and Brazil. The United States is known for producing high-quality cotton, particularly in states like Texas and California.'

In [ ]:
# Remembers
runnable.invoke(
    {"input": "What is the total value of cotton exports of this country"},
    config={"configurable": {"session_id": "sess1"}},
)

"In 2020, the United States exported around $5.9 billion worth of cotton. Cotton is a significant agricultural product for the United States, and it plays a crucial role in the country's economy."

In [ ]:
# New session_id --> does not remember.
runnable.invoke(
    {"input": "What is the total value of cotton exports of this country"},
    config={"configurable": {"session_id": "sess2"}},
)

"I'm sorry, but I don't have real-time data on specific countries' cotton exports. Would you like me to look up the information for you?"